# Numerical verification of some discrete boundary potential formulas
This notebook is supplementary material for 

"_Cloaking for random walks and a discrete potential theory_" by
Fernando Guevara Vasquez and Trent DeGiovanni

The idea here is to generate a random graph subdivided into boundary nodes $B$, exterior nodes $E$ and a region of interest $\Omega$ that is connected to the rest of the nodes via $\partial\Omega$. Then we test the different identities on appropriate solutions to the Dirichlet problem on a graph. All the tests here work by computing the difference between the left and right hand side of a particular equality and checking that we numerically a vector close to zero using the `@test` macro. If all the tests in a cell pass, then `Test Passed` is displayed in the cell output.

Note: in the paper we use $I$ instead of $E$, the different nomenclature is to avoid clashing with the identity `LinearAlgebra.I`.


## Graph construction

In [12]:
using LinearAlgebra, SparseArrays, Test

nB = 5; nE = 10; n∂Ω = 5; nΩ = 10 # select number of nodes

nV  = nB + nE + n∂Ω + nΩ
# define sets of nodes as indices
Ω = 1:nΩ
∂Ω = nΩ .+ (1:n∂Ω)
E = (nΩ + n∂Ω) .+ (1:nE)
B = (nΩ + n∂Ω + nE) .+ (1:nB)
V = 1:nV # all vertices
Vᵒ = E ∪ ∂Ω ∪ Ω # all the vertices minus the ones boundary conditions are imposed
println("""The sets are:
    Ω  = $Ω
    ∂Ω = $∂Ω
    E  = $E
    B  = $B
    V  = $V
    Vᵒ = $Vᵒ""")

# generate incidence matrix with conductivities that are uniform random (0,1)
# using Erdös-Renyi model
p = 0.5 # probability of getting a link between two nodes
A = spzeros(nV,nV)
for i=1:nV, j=1:i-1
    A[i,j] =  rand()*(rand()>p)
end
A = (A + A')/2

# get rid of connections between E and Ω
A[E,Ω] .= 0; A[Ω,E] .= 0

# get rid of connections between B and anything other than E
A[B,∂Ω ∪ Ω] .= 0; A[∂Ω ∪ Ω,B] .= 0

# check ∂Ω is connected to both Ω and E
@assert sum(A[∂Ω,Ω])>0 && sum(A[∂Ω,E])>0 # if error is raised, simply rerun cell to draw another graph

The sets are:
Ω  = 1:10
∂Ω = 11:15
E  = 16:25
B  = 26:30
V  = 1:30
Vᵒ = [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Laplacian and Green functions
Note the identities that we find are not affected by the sign of the Laplacian (i.e. whether we choose it to be a positive semidefinite or negative semidefinite matrix)

In [13]:
# ensures a matrix has zero row sum by adjusting diagonal elements
zerosum(A) = A - spdiagm(A*ones(size(A,1))) 
L = zerosum(A)  # negative semi-definite Laplacian
#L = -zerosum(A) # positive semi-definite graph Laplacian

# Green functions with 0 Dirichlet condition on B
G = zeros(nV,nV)
G[Vᵒ,Vᵒ] = inv(Matrix(L[Vᵒ,Vᵒ]));

## The exterior and interior Laplacians
Here the exterior $L_\alpha^+$ and interior $L_\alpha^-$ are defined such that they are discrete Laplacians and $L=L_\alpha^+ + L_\alpha^-$.

In [14]:
α = 0.3 # blending factor (anything in [0,1])

# exterior Laplacian (zero inside Ω)
Le = copy(L);
Le[Ω,:] .= 0; Le[:,Ω] .= 0
Le[∂Ω,∂Ω] .*= 1-α
Le = zerosum(Le);

# interior Laplacian (zero outside Ω, i.e. B∪E)
Li = copy(L);
Li[B∪E,:] .= 0; Li[:,B∪E] .=0 ;
Li[∂Ω,∂Ω] .*= α
Li = zerosum(Li);
@test norm(Li+Le-L) ≈ 0 atol=1e-10

Test Passed

## Trace operators and single, double layer potential operators
Here we define:
* Dirichlet trace operator $\gamma_0$ and the Neumann trace operators $\gamma_1^\pm$
* Single $\mathcal{S}$ and double layer $\mathcal{D}^\pm$ potential operators
* Indicator function $\mathbb{1}$ of a set

In [15]:
R = I(nV)[∂Ω,:] # restriction to ∂Ω
γ0 = R          # Dirichlet trace operator
γ1e = - R*Le    # exterior Neumann trace operator
γ1i =   R*Li    # interior Neumann trace operator

S  = G*γ0'      # single layer potential operator
De = G*γ1e'     # exterior double layer potential operator
Di = G*γ1i'     # interior double layer potential operator

com(C,D) = C*D - D*C        # matrix commutator

𝟙(X) = [ i ∈ X for i ∈ V];  # indicator function of the set X

𝟙 (generic function with 1 method)

## Interior reproduction formulas

In [16]:
# generate a u that solves the Dirichlet problem inside ∂Ω ∪ Ω
u = zeros(nV)
u[B] = randn(nB)
u[Vᵒ] = -L[Vᵒ,Vᵒ]\(L[Vᵒ,B]*u[B])

# check interior reproduction formulas:
@test norm(S*γ1e*u - De*γ0*u - u.*𝟙(∂Ω ∪ Ω)) ≈ 0 atol=1e-10
@test norm(S*γ1i*u - Di*γ0*u - u.*𝟙(Ω))      ≈ 0 atol=1e-10

# check formulas with commutators:
@test norm(( G*com(Le,R'*R)*u - u)[∂Ω ∪ Ω]) ≈ 0 atol=1e-10
@test norm((-G*com(Li,R'*R)*u - u)[Ω])      ≈ 0 atol=1e-10

Test Passed

## Continuity of flux for a harmonic function

In [17]:
@test norm((γ1e - γ1i)*u) ≈ 0 atol=1e-10

Test Passed

## Exterior reproduction formulas

In [18]:
# generate a v that solves teh Dirichlet problem outside  ∂Ω ∪ Ω
f = zeros(nV); f[Ω] = randn(nΩ); # forcing term
v = zeros(nV);
v[Vᵒ]=-L[Vᵒ,Vᵒ]\f[Vᵒ];

# check exterior reproduction formulas:
@test norm(-S*γ1e*v + De*γ0*v - v.*𝟙(E))      ≈ 0 atol=1e-10
@test norm(-S*γ1i*v + Di*γ0*v - v.*𝟙(E∪∂Ω))   ≈ 0 atol=1e-10

# check formulas with commutators:
@test norm((-G*com(Le,R'*R)*v - v)[E])        ≈ 0 atol=1e-10
@test norm(( G*com(Li,R'*R)*v - v)[E∪∂Ω])     ≈ 0 atol=1e-10

Test Passed

## Jump relations

In [19]:
@test norm((γ1e-γ1i)*S  + I) ≈ 0 atol=1e-10  # discontinuity of Neumann trace of single layer potential
@test norm(γ0*(De - Di) + I) ≈ 0 atol=1e-10  # discontinuity of Dirichlet trace of double layer potential
@test norm(γ1e*Di - γ1i*De)  ≈ 0 atol=1e-10  # continuity of Neumann trace of double layer potential

Test Passed

## Boundary Layer Operators

In [20]:
bS = γ0*S          # Single Layer Operator
bD = γ0*(De+Di)/2   # Double Layer Operator
bDa = (γ1e+γ1i)*S/2 # Adjoint Double Layer Operator
bH = γ1e*Di;        # Hypersingular operator

## Jump relations for boundary layer operators

In [21]:
# discontinuity of Neumann trace of single layer potential
@test norm(γ1e*S - ( -I/2 + bD' )) ≈ 0 atol=1e-10
@test norm(γ1i*S - (  I/2 + bD' )) ≈ 0 atol=1e-10

# discontinuity of Dirichlet trace of double layer potential
@test norm(γ0*De - ( -I/2 + bD  )) ≈ 0 atol=1e-10
@test norm(γ0*Di - (  I/2 + bD  )) ≈ 0 atol=1e-10

Test Passed

## Calderón projectors

In [22]:
C = [-bD     bS
     -bH     bD' ]
Pi = I/2 + C # interior Calderón projector
Pe = I/2 - C # exterior Calderón projector
@test norm(Pi*Pi - Pi) ≈ 0 atol=1e-10
@test norm(Pe*Pe - Pe) ≈ 0 atol=1e-10

Test Passed

## DtN map tests
Checks DtN map formula $\Lambda = (\frac{I}{2} + D') S^{-1}$

In [23]:
DtN1 = Li[∂Ω,∂Ω] - Li[∂Ω,Ω]*(Li[Ω,Ω]\Array(Li[Ω,∂Ω]))
DtN2 = (I/2+bD')*inv(bS) 
@test norm(DtN1 - DtN2) ≈ 0 atol=1e-10

NtD1 = pinv(DtN1)
NtD2 = (I-ones(nB,nB)/nB)*bS*pinv(I/2+bD')
@test norm(NtD1 - NtD2) ≈ 0 atol=1e-10

Test Passed

## Check invertibility of operators
These are based on the uniqueness of certain problems

In [24]:
@test rank(bS) == nB # BSO should be invertible (uniqueness Dir prob)
@test rank(-I/2 + bD) == nB # uniqueness exterior Neumann prob
@test rank(I/2 + bD) == nB-1 # interior Neumann prob up to constants

Test Passed